In [8]:
import open3d as o3d
import numpy as np
import os

def calculate_m3c2_distances(cloud1: o3d.geometry.PointCloud, cloud2: o3d.geometry.PointCloud) -> np.ndarray:
    """
    Calculate distances between corresponding points in two point clouds (M3C2-like approach).
    """
    # Use KDTree for nearest neighbor search to find corresponding points
    cloud2_tree = o3d.geometry.KDTreeFlann(cloud2)

    distances = []
    for point in cloud1.points:
        [_, idx, _] = cloud2_tree.search_knn_vector_3d(point, 1)
        if len(idx) == 0:
            distances.append(np.inf)  # Assign infinity if no neighbor is found
        else:
            nearest_point = np.asarray(cloud2.points)[idx[0]]
            distance = np.linalg.norm(np.asarray(point) - nearest_point)
            distances.append(distance)

    return np.array(distances)

# Define paths
source_dir = r"C:\Users\devsibi\PycharmProjects\pythonProject4\lidar\las_data\786901_2a_20180718_Mikro-Detail 100 clusters\786901_2a_20180718_Mikro-Detail\100"
target_dir = r"C:\Users\devsibi\PycharmProjects\pythonProject4\lidar\las_data\786901_2a_20201118_Mikro-Detail 100 clusters\786901_2a_20201118_Mikro-Detail\100"
output_dir = r"C:\Users\devsibi\PycharmProjects\pythonProject4\lidar\output files\streaming Result\4"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Initialize total points filtered counter
total_filtered_points = 0

# Loop through source clusters (from 1 to 99) and target clusters (from 0 to 98)
for i in range(1, 100):  # source clusters start from 1, so loop from 1 to 99
    source_file = f"cluster_{i}.ply"  # source files start from cluster_1.ply
    target_file = f"cluster_{i-1}.ply"  # target files start from cluster_0.ply

    source_path = os.path.join(source_dir, source_file)
    target_path = os.path.join(target_dir, target_file)

    print(f"Processing source cluster {i} with target cluster {i-1}...")

    if os.path.exists(source_path) and os.path.exists(target_path):
        # Load the point clouds
        source_cloud = o3d.io.read_point_cloud(source_path)
        target_cloud = o3d.io.read_point_cloud(target_path)

        if len(source_cloud.points) == 0 or len(target_cloud.points) == 0:
            print(f"Empty point cloud for cluster {i}. Skipping.")
            continue

        # Calculate distances between corresponding points
        distances = calculate_m3c2_distances(source_cloud, target_cloud)

        # Define the threshold value
        threshold = 0.03

        # Find indices of points with distances greater than the threshold
        indices_above_threshold = np.where(distances > threshold)[0]

        # Create a new point cloud with only the points above the threshold
        filtered_points = np.asarray(source_cloud.points)[indices_above_threshold]
        filtered_colors = np.asarray(source_cloud.colors)[indices_above_threshold] if source_cloud.colors else None

        filtered_point_cloud = o3d.geometry.PointCloud()
        filtered_point_cloud.points = o3d.utility.Vector3dVector(filtered_points)
        if filtered_colors is not None:
            filtered_point_cloud.colors = o3d.utility.Vector3dVector(filtered_colors)

        # Save the filtered point cloud
        filtered_point_cloud_file_path = os.path.join(output_dir, f"filtered_cluster_{i}.ply")
        o3d.io.write_point_cloud(filtered_point_cloud_file_path, filtered_point_cloud)

        # Update total points filtered
        total_filtered_points += len(filtered_points)

        # Print the number of points filtered
        print(f"Cluster {i}: {len(filtered_points)} points filtered. Saved to {filtered_point_cloud_file_path}")

    else:
        print(f"Missing source or target point cloud for cluster {i}")

# Print the total number of points filtered
print(f"Total number of points filtered across all clusters: {total_filtered_points}")
print("Processing complete.")


Processing source cluster 1 with target cluster 0...
Cluster 1: 716233 points filtered. Saved to C:\Users\devsibi\PycharmProjects\pythonProject4\lidar\output files\streaming Result\4\filtered_cluster_1.ply
Processing source cluster 2 with target cluster 1...
Cluster 2: 806333 points filtered. Saved to C:\Users\devsibi\PycharmProjects\pythonProject4\lidar\output files\streaming Result\4\filtered_cluster_2.ply
Processing source cluster 3 with target cluster 2...
Cluster 3: 188816 points filtered. Saved to C:\Users\devsibi\PycharmProjects\pythonProject4\lidar\output files\streaming Result\4\filtered_cluster_3.ply
Processing source cluster 4 with target cluster 3...
Cluster 4: 44882 points filtered. Saved to C:\Users\devsibi\PycharmProjects\pythonProject4\lidar\output files\streaming Result\4\filtered_cluster_4.ply
Processing source cluster 5 with target cluster 4...
Cluster 5: 114911 points filtered. Saved to C:\Users\devsibi\PycharmProjects\pythonProject4\lidar\output files\streaming Res